<a href="https://colab.research.google.com/github/myeze/MachineLearningModels/blob/main/CarEvaluationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Evaluation Neural Network Model

**This notebook contains a model created by Myles Ezeanii.**

Throughout the notebook, I was able to form a Neural Network Model used to predict the value of comparative value of motor vehichles using a preexisting database. The features included are:

---
* **Buying Price**
  * vhigh, high, med, low.
* **Maintenance Price**
  * vhigh, high, med, low
* **Number of Doors**
  * 2, 3, 4, 5, more
* **Seating Capacity**
  *  2, 4, more
* **Luggage Boot Size**
  * small, med, big
* **Car Safety (estimated)**
  *  low, med, high
---

The "goal" field represents the evaulation level of our vehichle

This is represented as an categorical value in the form of: (unacceptable, acceptable, good, very good).

---

The overall goal is for the model to accuratly determine the worth a car has in respect to others that have been seen.

---
Bohanec, M. (1988). Car Evaluation [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5JP48.

In [ ]:
!pip install ucimlrepo
!pip install tensorflow

In [4]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
#print(car_evaluation.metadata)

# variable information
#print(car_evaluation.variables)
print(X)
print(y)

     buying  maint  doors persons lug_boot safety
0     vhigh  vhigh      2       2    small    low
1     vhigh  vhigh      2       2    small    med
2     vhigh  vhigh      2       2    small   high
3     vhigh  vhigh      2       2      med    low
4     vhigh  vhigh      2       2      med    med
...     ...    ...    ...     ...      ...    ...
1723    low    low  5more    more      med    med
1724    low    low  5more    more      med   high
1725    low    low  5more    more      big    low
1726    low    low  5more    more      big    med
1727    low    low  5more    more      big   high

[1728 rows x 6 columns]
      class
0     unacc
1     unacc
2     unacc
3     unacc
4     unacc
...     ...
1723   good
1724  vgood
1725  unacc
1726   good
1727  vgood

[1728 rows x 1 columns]


In [5]:
# Split the data into training and testing sets
XTrained, XTested, yTrained, yTested = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Create an imputer to replace missing values with the mean
imputer = SimpleImputer(strategy='mean')

In [7]:
# Separate the categorical and numerical features
categoryfeatures = X.select_dtypes(include=['object']).columns.tolist()
numericalfeatures = X.select_dtypes(exclude=['object']).columns.tolist()

In [8]:
# Create transformers for numerical and categorical features
numericalTransformer = SimpleImputer(strategy='mean') # Use mean for numerical features
categoryTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Use most frequent for categorical features
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')) # One-hot encode categorical features
])

In [9]:
# Create a ColumnTransformer to apply transformers to the correct columns
preprocessor = ColumnTransformer(transformers=[
    ('num', numericalTransformer, numericalfeatures),
    ('cat', categoryTransformer, categoryfeatures)
])

In [10]:
# Fit the preprocessor on the training data and transform both training and testing data
XTrainedImputed = preprocessor.fit_transform(XTrained)
XTestedImputed = preprocessor.transform(XTested)

In [11]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(XTrainedImputed.shape[1],)), # Input layer with 6 features
    keras.layers.Dense(64, activation='relu'), # Hidden layer with 64 units and ReLU activation
    keras.layers.Dense(4, activation='softmax') # Output layer with 4 units (for 4 classes) and softmax activation
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Create a LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder on the target variable and transform it
yTrainedEncoded = label_encoder.fit_transform(yTrained.values.ravel())

# Now use yTrainedEncoded in model.fit
model.fit(XTrainedImputed, yTrainedEncoded, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6112 - loss: 1.0486 - val_accuracy: 0.6259 - val_loss: 0.7256
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7247 - loss: 0.5634 - val_accuracy: 0.7698 - val_loss: 0.5111
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8803 - loss: 0.3562 - val_accuracy: 0.8705 - val_loss: 0.3671
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8786 - loss: 0.2833 - val_accuracy: 0.8849 - val_loss: 0.2783
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9260 - loss: 0.2088 - val_accuracy: 0.9209 - val_loss: 0.2221
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9441 - loss: 0.1585 - val_accuracy: 0.9784 - val_loss: 0.1729
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9624 - loss: 0.1443 - val_accuracy: 0.9496 - val_loss: 0.1568
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9773 - loss: 0.0920 - val_accuracy: 0.9568 - val_loss

In [14]:
# Assuming label_encoder is already defined from previous code
yTestedEncoded = label_encoder.transform(yTested.values.ravel())
loss, accuracy = model.evaluate(XTestedImputed, yTestedEncoded, verbose=0)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Test Loss: 0.10376929491758347
Test Accuracy: 0.97398841381073
